In [2]:
%load_ext cython

## Calling Python from C

In order to generate a `.c` and `.h` file to use externally (instead of just the `.pyx` file), we need to add the ketword **public** to our cdef declarations.

Here is an example of a pyrex file to be used. The distutils comment can go in the pyrex file.

In [3]:
%%cython
#distutils: language=c++

cdef public float PI = 3.1415926

cpdef public double getPi():
    print("Getting PI is being called.")
    return PI

cdef public double getE():
    print("Getting E")
    return 2.718281828

The following function does not work in C++. The whole process is an embarassment. Cython is not suitable to extend C. Cython can extend Cython with C capabilities however.

In [15]:
getPi()

Getting PI is being called.


3.141592502593994

In [17]:
!python CythonBuild/setup.py build_ext --inplace

Compiling CythonBuild/PythonFromC.pyx because it changed.

D:\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: D:\Programming\Python\Cython\CythonBuild\PythonFromC.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)



[1/1] Cythonizing CythonBuild/PythonFromC.pyx
running build_ext
building 'PythonFromC' extension
D:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -ID:\Anaconda3\include -ID:\Anaconda3\include "-ID:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\ATLMFC\include" "-ID:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\include" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\winrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\cppwinrt" /EHsc /TpCythonBuild/PythonFromC.cpp /Fobuild\temp.win-amd64-3.8\Release\CythonBuild/P

Make sure the PYTHONPATH and PYTHONHOME environment vairables are set correctly. Next we need the Python header file `Python.h` located in PYTHONPATH/include/Python.h. Add a input folder PYTHONPATH/libs. Add the extracted file to the C++ project.

## Calling C libraries from Cython

Recall that a header file can be read in as external file and the required functions can be copied over.

In [8]:
%%cython

cdef extern from "stdlib.h":
    void* malloc(size_t size)
    void free(void *ptr)

cpdef list pyBubbleSort(list x):
    cdef:
        int *array
        int i, j, N, placeholder
    
    N = len(x)
    
    array = <int*>malloc(sizeof(int) * N)
    if array == NULL:
        raise MemoryError("Unable to allocate array")
    
    for i in range(N):
        array[i] = x[i]
    
    for i in range(N-1, 0, -1):
        for j in range(i):
            if array[j] > array[j+1]:
                placeholder = array[j]
                array[j] = array[j+1]
                array[j+1] = placeholder
    
    for i in range(N):
        x[i] = array[i]
    free(array)
    
    return x
    

In [6]:
pyBubbleSort([2, 1, 4, 5, 0])

[0, 1, 2, 4, 5]

In [9]:
%timeit pyBubbleSort([2, 1, 4, 5, 0])

370 ns ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%%cython

cdef extern from 'stdlib.h':
    # @param - base = C style array to be sorted
    # @param - N = length of array
    # @param - size = size in bytes of elements in array
    void qsort (void* base, size_t num, size_t size,
            int (*compar)(const void*,const void*))
    void* malloc(size_t size)
    void free(void *ptr)

cdef int intCompare(const void *a, const void *b):
    cdef int ia, ib
    ia = (<int*>a)[0]
    ib = (<int*>b)[0]
    return ia - ib

cdef int reverseIntCompare(const void *a, const void *b):
    return -intCompare(a, b)

ctypedef int (*qsortCompare)(const void *, const void *)

def pyqsort(list x, reverse=False):
    cdef qsortCompare compare_callback
    
    if reverse:
        compare_callback = reverseIntCompare
    else:
        compare_callback = intCompare
        
    cdef:
        int *array
        int i, N
    
    N = len(x)
    
    array = <int*>malloc(sizeof(int) * N)
    if array == NULL:
        raise MemoryError("Unable to allocate array")
    
    for i in range(N):
        array[i] = x[i]

    qsort(array, N, sizeof(int), compare_callback)

    for i in range(N):
        x[i] = array[i]
    free(array)
    
    return x

In [5]:
pyqsort([2, 1, 4, 5, 0])

[0, 1, 2, 4, 5]

In [6]:
pyqsort([2, 1, 4, 5, 0], True)

[5, 4, 2, 1, 0]

In [7]:
%timeit pyqsort([2, 1, 4, 5, 0])

541 ns ± 84.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
